In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install wordcloud

import matplotlib.pyplot as plt
import matplotlib as mpl

import re #регулярки для проверки на латиницу
import string
import numpy as np
import itertools
from collections import defaultdict
from operator import itemgetter
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
punctuation =string.punctuation +'–'
#короткое тире - часть слова
re_punctuation = '!"#$%&\'()*+,./:;<=>?@~–^_`{|}—»«'
numbers = '0123456789'

In [5]:
class meaningless_words:
    def __init__(self, russian_stopwords = stopwords.words('russian'), english_stopwords = stopwords.words('english')):
        self.universal = set(russian_stopwords) | set(english_stopwords )
        self.russian = set(russian_stopwords)
        self.english = set(english_stopwords)

In [6]:
class LanguageException(BaseException):
    def __init__(self, problem = ''):
        self.problem = problem
    def __str__(self):
        return str(self.problem)

In [7]:
class Language:
    def __init__(self, *args):
        self.universal = 'universal'
        self.russian = 'russian'
        self.english = 'english'
        self.сyrillic = 'russian'
        self.latin = 'english'
        self.other = args

In [8]:
my_meaningless_words = meaningless_words()
language = Language()

In [9]:
def get_the_words(text):
    
    only_words = re.sub('[' +re_punctuation + '\n' + numbers +']', ' ', text.lower())
    only_words = re.sub('\s-+\s', ' ', only_words)
    return only_words


def get_punctuation_marks(text):
    marks = re.findall('[' + re_punctuation + ']', text)
    return ''.join(marks)

def separate_languages(words):
    rus = re.compile('^[а-яА-Я]+')
    eng = re.compile('^[a-zA-Z]+')
    
    сyrillic = list(filter(rus.match, words))
    latin = list(filter(eng.match, words))
    return сyrillic, latin

#число восклицательных и вопросительных знаков
def calculate_emphasis(text):
    еxclamation_points = re.findall('[!]', text)
    question_marks = re.findall('[?]', text)
    return (len(еxclamation_points), len(question_marks))

In [10]:
class Text_information:
    def __init__(self, text=''):
        self.text = text
        self.words = word_tokenize(get_the_words(text))
        self.marks = word_tokenize(get_punctuation_marks(text))
        сyrillic, latin = separate_languages(self.words)
        self.is_сyrillic = bool(сyrillic)
        self.сyrillic = сyrillic
        self.is_latin = bool(latin)
        self.latin = latin       
        self.emphasis = calculate_emphasis(text)
        self.marks_statistics = FreqDist(self.marks)
    
    def word_statistics(self,meaningful = False,language = 'universal'): 
        if not meaningful:
            if language == 'universal':
                return FreqDist(self.words)
            if language == 'russian':
                if not self.is_сyrillic:
                    raise LanguageException('No сyrillic words in the text')
                return FreqDist(self.сyrillic)
            if language == 'english':
                if not self.is_latin:
                    raise LanguageException('No latin words in the text')
                return FreqDist(self.latin)
            raise LanguageException('Language is not supported')
        
        return FreqDist(self.only_meaningful_words( language))
        
    def plot_word_statistic(self, number = 1, meaningful = False, language = 'universal'):
        self.word_statistics(meaningful, language).plot(number)
        

        
    def deviation_from_Zipf(self, meaningful = False, language = 'universal'):
        frequencies = sorted(list(self.word_statistics(meaningful,language).values()), reverse = True)
        frequencies = np.array(frequencies)
        hypothesis = 1 / (np.arange(frequencies.size)+1) * frequencies[0]
        #print(frequencies)
        return np.std(frequencies - hypothesis)
        
    def plot_with_Zipf(self, meaningful = False, language = 'universal'):

        frequencies = sorted(list(self.word_statistics(meaningful, language).values()), reverse = True)

        hypothesis = 1 / (np.arange(len(frequencies))+1) * frequencies[0]
        print('hypothesis:' ,hypothesis)
        
        fig, ax = plt.subplots()
        plt.plot(range(1,len(frequencies)+1), frequencies)
        plt.plot(range(1, len(frequencies)+1), hypothesis)
        ax.set_xlabel('log порядкового номера слова')
        ax.set_ylabel('log частоты')
        plt.xscale('log')
        plt.yscale('log')
        plt.plot()
        
    def only_meaningful_words(self, language = 'universal'):
            
        if language == 'universal':
            return [x for x in self.words if x not in my_meaningless_words.universal]
        elif language == 'russian':
            if not self.is_сyrillic:
                raise LanguageException('No сyrillic words in the text')
            return [x for x in self.сyrillic if x not in my_meaningless_words.russian]
        elif language == 'english':
            if not self.is_latin:
                raise LanguageException('No latin words in the text')
            return [x for x in self.latin if x not in my_meaningless_words.english]
        else: 
            raise LanguageException('Language is not supported')    
            
    
    def percent_of_unique_words(self, meaningful = False, language = 'universal'):
        D = self.some_statistic(meaningful, language)
        return D['number_of_unique_words']/D['number_of_words'] *100
    
    def percent_of_meaningful_words(self, language = 'universal'):
        return (self.some_statistic(meaningful = True, language=language)['number_of_words'] / 
    self.some_statistic(meaningful = False, language=language)['number_of_words'] *100)
  
    def number_of_words_occurring_n_times(self, meaningful = False,language = 'universal'):
        Dict = self.word_statistics(meaningful,language)
        New_Dict = defaultdict(int)
        for value, items in itertools.groupby(sorted(Dict.items(), key=itemgetter(1)), key=itemgetter(1)):
            New_Dict[value] = set(map(itemgetter(0), items))
        return New_Dict

    def plot_the_number_of_words_by_occurrence(self, meaningful = False,language = 'universal', logarithmic = False):
        Dict = self.number_of_words_occurring_n_times(meaningful,language)
        frequencies = []
        quantities = []
        
        for x in sorted(Dict.keys()):
            frequencies.append(x)
            quantities.append(len(Dict[x]))
        
        #print(quantities)
        fig, ax = plt.subplots()
        plt.plot(frequencies, quantities)
        ax.set_xlabel('частота встречаемости слова')
        ax.set_ylabel('количество слов данной частоты')
        if logarithmic:
            plt.xscale('log')
            plt.yscale('log')
        plt.plot()


    def some_statistic(self, meaningful = False, language = 'universal'):
        Dict = self.word_statistics(meaningful, language)
        frequencies = list(Dict.values())

        frequencies = np.array(frequencies)
        F = {}
        F['most_frequent'] = frequencies.max()
        F['least_frequent'] = frequencies.min()
        F['number_of_words'] = frequencies.sum()
        F['number_of_different_words'] = frequencies.size
        F['mean'] = np.mean(frequencies)
        F['dispersion'] = np.std(frequencies)
        F['median'] = np.median(frequencies)
        F['number_of_unique_words'] = np.count_nonzero(frequencies == 1)
        return F
    
    def percent_of_most_common_words(self,k = 20, meaningful = False, language = 'universal' ):
        if k > 100 or k < 0:
            raise LanguageException('it\'s not a percentage')

        frequencies = sorted(list(self.word_statistics(meaningful,language).values()), reverse = True)
          
        s = 0
        for e in range(int(len(frequencies) * k / 100)):
            s = s+ frequencies[e]
        return (s / sum(frequencies)) * 100
    
    def phrases(self, length = 2, language = 'universal'):
        if length == 1:
            return self.words
        
        if language == 'universal':
            prewords = self.words
        elif language == 'russian':
            if not self.is_сyrillic:
                raise LanguageException('No сyrillic words in the text')
            prewords = self.сyrillic
        elif language == 'english':
            if not self.is_latin:
                raise LanguageException('No latin words in the text')
            prewords = self.latin
        else: 
            raise LanguageException('Language is not supported') 
        
        word_groups = list(zip(*(prewords[i:-1] for i in range(length))))
        return word_groups
    
    def phrases_statistics(self, length = 2, language = 'universal'):
        if length==1:
            return self.word_statistics(False, language)
        return(FreqDist(self.phrases(length, language)))
    
    def plot_phrases_with_Zipf(self,length = 2, language = 'universal'):
        if length ==1:
            self.plot_with_Zipf(False, language)
        else:
            frequencies = sorted(list(self.phrases_statistics(length,language).values()), reverse = True)
            hypothesis = 1 / (np.arange(len(frequencies))+1) * frequencies[0]
            fig, ax = plt.subplots()
            plt.plot(range(1,len(frequencies)+1), frequencies)
            plt.plot(range(1, len(frequencies)+1), hypothesis)
            ax.set_xlabel('log порядкового номера слова')
            ax.set_ylabel('log частоты')
            plt.xscale('log')
            plt.yscale('log')
            plt.plot()

            
    def some_phrases_statistic(self, length = 2, language = 'universal'):
        if length ==1:
            return self.some_statistic(False, language) 
        Dict = self.phrases_statistics(length, language)
        frequencies = list(Dict.values())
        frequencies = np.array(frequencies)
        F = {}
        F['most_frequent'] = frequencies.max()
        F['least_frequent'] = frequencies.min()
        F['number_of_words'] = frequencies.sum()
        F['number_of_different_words'] = frequencies.size
        F['mean'] = np.mean(frequencies)
        F['dispersion'] = np.std(frequencies)
        F['median'] = np.median(frequencies)
        F['number_of_unique_words'] = np.count_nonzero(frequencies == 1)
        return F
    
    def plot_phrases_statistic(self,number = 1,length = 2, language = 'universal'):
        if length ==1:
             self.plot_word_statistic(number, False, language)
        else:
            self.phrases_statistics(length, language).plot(number)
    
    def number_of_phrases_occurring_n_times(self,length = 2, language = 'universal'):
        if length ==1:
            return self.number_of_words_occurring_n_times( meaningful = False,language=language)
        Dict = self.phrases_statistics(length,language)
        New_Dict = {}
        for value, items in itertools.groupby(sorted(Dict.items(), key=itemgetter(1)), key=itemgetter(1)):
            New_Dict[value] = set(map(itemgetter(0), items))
        return New_Dict    
    
    
    def plot_the_number_of_phrases_by_occurrence(self, length = 2, language = 'universal', logarithmic = False):
        if length ==1:
            self.plot_the_number_of_words_by_occurrence(meaningful = False, language=language, logarithmic=logarithmic)
        else:
            Dict = self.number_of_phrases_occurring_n_times(length,language)
            frequencies = []
            quantities = []
        
            for x in sorted(Dict.keys()):
                frequencies.append(x)
                quantities.append(len(Dict[x]))
        
            #print(quantities)
            fig, ax = plt.subplots()
            plt.plot(frequencies, quantities)
            ax.set_xlabel('частота встречаемости слова')
            ax.set_ylabel('количество слов данной частоты')
            if logarithmic:
                plt.xscale('log')
                plt.yscale('log')
            plt.plot()
            
            
    def cloud(self,  meaningful = False,language = 'universal' ):
        if not meaningful:
            if language == 'universal':
                wcloud = WordCloud().generate(' '.join(self.words))
            elif language == 'russian':
                if not self.is_сyrillic:
                    raise LanguageException('No сyrillic words in the text')
                wcloud = WordCloud().generate(' '.join(self.сyrillic))
            elif language == 'english':
                if not self.is_latin:
                    raise LanguageException('No latin words in the text')
                wcloud = WordCloud().generate(' '.join(self.latin))
            else: 
                raise LanguageException('Language is not supported') 
                
            
        else:
            wcloud = WordCloud().generate(' '.join(self.only_meaningful_words(language)))
            
        plt.imshow(wcloud  , interpolation='bilinear')
        plt.axis('off')
        plt.show()